# Page Rank comparison

In the following, we provide the python implementation of the page rank algorithm used as reference for testing the correctness and robustness of our custom Prolog Page Rank implementation 

## Custom NetworkX Page Rank implementation

The only change w.r.t. official standard implementation is that the stopping criterion is **not** multiplied by the total number of nodes when checking for convergence

In [2]:
import networkx as nx

def custom_pagerank_scipy(
    G,
    alpha=0.85,
    personalization=None,
    max_iter=100,
    tol=1.0e-6,
    nstart=None,
    weight="weight",
    dangling=None,
):
    """Returns the PageRank of the nodes in the graph.

    PageRank computes a ranking of the nodes in the graph G based on
    the structure of the incoming links. It was originally designed as
    an algorithm to rank web pages.

    Parameters
    ----------
    G : graph
      A NetworkX graph.  Undirected graphs will be converted to a directed
      graph with two directed edges for each undirected edge.

    alpha : float, optional
      Damping parameter for PageRank, default=0.85.

    personalization: dict, optional
      The "personalization vector" consisting of a dictionary with a
      key some subset of graph nodes and personalization value each of those.
      At least one personalization value must be non-zero.
      If not specified, a nodes personalization value will be zero.
      By default, a uniform distribution is used.

    max_iter : integer, optional
      Maximum number of iterations in power method eigenvalue solver.

    tol : float, optional
      Error tolerance used to check convergence in power method solver.
      The iteration will stop after a tolerance of ``len(G) * tol`` is reached.

    nstart : dictionary, optional
      Starting value of PageRank iteration for each node.

    weight : key, optional
      Edge data key to use as weight.  If None weights are set to 1.

    dangling: dict, optional
      The outedges to be assigned to any "dangling" nodes, i.e., nodes without
      any outedges. The dict key is the node the outedge points to and the dict
      value is the weight of that outedge. By default, dangling nodes are given
      outedges according to the personalization vector (uniform if not
      specified) This must be selected to result in an irreducible transition
      matrix (see notes under google_matrix). It may be common to have the
      dangling dict to be the same as the personalization dict.

    Returns
    -------
    pagerank : dictionary
       Dictionary of nodes with PageRank as value

    Examples
    --------
    >>> from networkx.algorithms.link_analysis.pagerank_alg import _pagerank_scipy
    >>> G = nx.DiGraph(nx.path_graph(4))
    >>> pr = _pagerank_scipy(G, alpha=0.9)

    Notes
    -----
    The eigenvector calculation uses power iteration with a SciPy
    sparse matrix representation.

    This implementation works with Multi(Di)Graphs. For multigraphs the
    weight between two nodes is set to be the sum of all edge weights
    between those nodes.

    See Also
    --------
    pagerank

    Raises
    ------
    PowerIterationFailedConvergence
        If the algorithm fails to converge to the specified tolerance
        within the specified number of iterations of the power iteration
        method.

    References
    ----------
    .. [1] A. Langville and C. Meyer,
       "A survey of eigenvector methods of web information retrieval."
       http://citeseer.ist.psu.edu/713792.html
    .. [2] Page, Lawrence; Brin, Sergey; Motwani, Rajeev and Winograd, Terry,
       The PageRank citation ranking: Bringing order to the Web. 1999
       http://dbpubs.stanford.edu:8090/pub/showDoc.Fulltext?lang=en&doc=1999-66&format=pdf
    """
    import numpy as np
    import scipy as sp
    import scipy.sparse  # call as sp.sparse

    N = len(G)
    if N == 0:
        return {}

    nodelist = list(G)
    A = nx.to_scipy_sparse_array(G, nodelist=nodelist, weight=weight, dtype=float)
    S = A.sum(axis=1)
    S[S != 0] = 1.0 / S[S != 0]
    # TODO: csr_array
    Q = sp.sparse.csr_array(sp.sparse.spdiags(S.T, 0, *A.shape))
    A = Q @ A

    # initial vector
    if nstart is None:
        x = np.repeat(1.0 / N, N)
    else:
        x = np.array([nstart.get(n, 0) for n in nodelist], dtype=float)
        x /= x.sum()

    # Personalization vector
    if personalization is None:
        p = np.repeat(1.0 / N, N)
    else:
        p = np.array([personalization.get(n, 0) for n in nodelist], dtype=float)
        if p.sum() == 0:
            raise ZeroDivisionError
        p /= p.sum()
    # Dangling nodes
    if dangling is None:
        dangling_weights = p
    else:
        # Convert the dangling dictionary into an array in nodelist order
        dangling_weights = np.array([dangling.get(n, 0) for n in nodelist], dtype=float)
        dangling_weights /= dangling_weights.sum()
    is_dangling = np.where(S == 0)[0]

    # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = alpha * (x @ A + sum(x[is_dangling]) * dangling_weights) + (1 - alpha) * p
        # check convergence, l1 norm
        err = np.absolute(x - xlast).sum()
        if err < tol:
            return dict(zip(nodelist, map(float, x)))
    raise nx.PowerIterationFailedConvergence(max_iter)

## Upload GraphBrain instances in list-based formalism

Simply run the below cell and select the GraphBrain exported instances after they have been translated in the list-based formalism by the KBRestruturer java class

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving list_exportedGraph.pl to list_exportedGraph.pl
User uploaded file "list_exportedGraph.pl" with length 108654977 bytes


## Scan the whole file to identify nodes and arcs

In [4]:
import re

# change this if the file uploaded has a different name
FILE_NAME = "list_exportedGraph.pl"

nodes = []
arcs = []

with open(FILE_NAME, "r", encoding='utf-8') as f:
    for line in f:
        match = re.match("node_properties\\(([0-9]+),\\s*\[(.+)]\\)\\.", line)
        if match:
            nodes.append(match.group(1))

        match = re.match("arc\\(([0-9]+),\\s*([0-9]+),\\s*([0-9]+)\\)\\.", line)
        if match:
            arcs.append((match.group(2), match.group(3)))


## Create graph

In [5]:
graph = nx.DiGraph()

for node in nodes:
    graph.add_node(int(node))

for (node_0, node_1) in arcs:
    graph.add_edge(int(node_0), int(node_1))

## Run classical page rank

In [6]:
page_rank_dict = custom_pagerank_scipy(
    graph,
    alpha=0.85,
    max_iter=100
)

print("Page Rank for nodes 0, 1, 457:")
print(f"0 - {page_rank_dict[0]}")
print(f"1 - {page_rank_dict[1]}")
print(f"457 - {page_rank_dict[457]}")

Page Rank for nodes 0, 1, 457:
0 - 8.919201525861478e-07
1 - 2.028072967268872e-06
457 - 1.1809186304811592e-06


## Run Page Rank with custom start vector and personalization vector

In [7]:
page_rank_dict = custom_pagerank_scipy(
    graph,
    alpha=0.85,
    max_iter=100,
    tol=0.000001,
    nstart={0: 5.0, 2: 3.2, 6: 2.4},
    personalization={15: 3.1, 67: 2.3, 457: 0.5}
)

print("Page Rank for nodes 0, 1, 457:")
print(f"0 - {page_rank_dict[0]}")
print(f"1 - {page_rank_dict[1]}")
print(f"457 - {page_rank_dict[457]}")

Page Rank for nodes 0, 1, 457:
0 - 0.0
1 - 0.0
457 - 0.024468298460878633
